<a href="https://colab.research.google.com/github/NivethikaSakthivel/AI-ML-Internship-Assignment-2/blob/master/ML_Intern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask ngrok gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 

In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken 2mYBRRtnrtiqrmd5IbW0548KiDB_7NxKsnp6s3mXrM9fofGRq

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify
from threading import Thread
from pyngrok import ngrok

app = Flask(__name__)

@app.route('/search_flights', methods=['GET'])
def search_flights():
    origin = request.args.get('origin')
    destination = request.args.get('destination')
    date = request.args.get('date')

    flights = [
        {"flight_id": 1, "airline": "Airways A", "departure_time": "10:00 AM", "arrival_time": "1:00 PM", "price": "$150"},
        {"flight_id": 2, "airline": "Airways B", "departure_time": "2:00 PM", "arrival_time": "5:00 PM", "price": "$180"}
    ]

    return jsonify({"flights": flights})

@app.route('/book_flight', methods=['POST'])
def book_flight():
    flight_id = request.json.get('flight_id')

    return jsonify({"status": "success", "message": "Your flight has been booked!", "booking_reference": "ABC123"})

# Run Flask app in a background thread
def run_flask():
    app.run()

flask_thread = Thread(target=run_flask)
flask_thread.start()

# Expose the Flask app via ngrok
public_url = ngrok.connect(5000)
print(f"Flask app is running at {public_url}")

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Flask app is running at NgrokTunnel: "https://430f-34-46-81-119.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
import gradio as gr
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

API_URL = public_url

def search_flights(origin, destination, date):
    response = requests.get(f"{API_URL}/search_flights", params={"origin": origin, "destination": destination, "date": date})
    flights = response.json().get('flights', [])

    if flights:
        return flights
    else:
        return []

def book_flight(flight_id):
    response = requests.post(f"{API_URL}/book_flight", json={"flight_id": flight_id})
    return response.json()

def send_email(recipient, subject, body):
    sender_email = "prasanndhraajumr.22cse@kongu.edu"
    sender_password = "25-Aug-04"

    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = recipient
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        text = message.as_string()
        server.sendmail(sender_email, recipient, text)
        server.quit()
    except Exception as e:
        print(f"Failed to send email: {str(e)}")

def chatbot(input_text, state):
    if state is None:
        state = {}

    if "booking" in state:
        flight_id = state.get("selected_flight_id")
        booking_response = book_flight(flight_id)

        if booking_response.get('status') == "success":
            booking_reference = booking_response.get('booking_reference')
            state["step"] = "booked"
            send_email("customer@example.com", "Booking Confirmation", f"Your booking reference is {booking_reference}.")
            return "Your flight has been booked! A confirmation email has been sent to you.", state

    if state.get("step") == "selecting_flight":
        try:
            selected_flight = int(input_text.strip())
            if selected_flight > 0 and selected_flight <= len(state["flights"]):
                state["selected_flight_id"] = selected_flight
                state["booking"] = True
                return f"Booking flight {selected_flight}...", state
            else:
                return "Invalid selection. Please choose a valid flight number.", state
        except ValueError:
            return "Please enter a valid flight number.", state

    if "step" not in state:
        state["step"] = "start"
        return "Welcome! Please provide your travel details (e.g., 'New York to Los Angeles on October 15th').", state

    try:
        parts = input_text.lower().split(' to ')
        origin = parts[0].strip().title()
        destination_date = parts[1].strip().split(' on ')
        destination = destination_date[0].strip().title()
        date = destination_date[1].strip().title()

        flights = search_flights(origin, destination, date)
        state["flights"] = flights
        state["step"] = "selecting_flight"

        if flights:
            response_text = "Here are the available flights:\n"
            for idx, flight in enumerate(flights, start=1):
                response_text += f"Flight {idx}: {flight['airline']} - {flight['departure_time']} to {flight['arrival_time']} - {flight['price']}\n"
            response_text += "Please select a flight by typing the flight number."
            return response_text, state
        else:
            return "No flights found for the given route and date. Please try again.", state

    except IndexError:
        return "Could not understand your input. Please provide travel details in the format 'Origin to Destination on Date'.", state
iface = gr.Interface(
    fn=chatbot,
    inputs=["text", "state"],
    outputs=["text", "state"],
    title="Automated Ticket Booking System",
    description="A chatbot for booking flight tickets.",
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e59b0f864d5f6f0bd9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
